### Montagem do Google Drive

In [ ]:
# Importar bibliotecas necessárias para manipulação de arquivos
import os
import shutil

# Definir o caminho de montagem do Google Drive
mount_path = '/content/drive'

# Montar o Google Drive
from google.colab import drive
drive.mount(mount_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install PyWavelets librosa pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 40.0 MB/s eta 0:00:00


### Importação de Bibliotecas e Configuração Inicial

In [ ]:
# Importando bibliotecas essenciais
import os
import librosa
import librosa.display
import pywt
import numpy as np
import matplotlib.pyplot as plt
import gc  # Gerenciamento manual de memória
from tqdm import tqdm  # Barra de progresso para loops

# Definindo caminhos principais
base_path = "/content/drive/MyDrive/FoR/for-2sec/for-2seconds"
output_base_path = "/content/drive/MyDrive/Datasets_Scalograms/CMorl"
# output_base_path = "/content/drive/MyDrive/Datasets_Scalograms/Morl"
# output_base_path = "/content/drive/MyDrive/Datasets_Scalograms/MexicanHat"
os.makedirs(output_base_path, exist_ok=True)

# Definindo categorias e subcategorias da base de dados
categories = ['training', 'testing', 'validation']
subcategories = ['fake', 'real']

# Configurações globais
sampling_rate = 16000         # Taxa de amostragem uniforme
target_size = (224, 224)       # Tamanho alvo das imagens em pixels

# Família de wavelet utilizada
wavelet = 'cmor1.5-1.0'
# wavelet = 'morl'
# wavelet = 'mxhat'


### Função para Processar e Salvar Escalogramas

In [ ]:
def process_and_save_scalogram(audio_path, output_path):

    # Verificando se o arquivo de saída já existe
    if os.path.exists(output_path):
        return

    try:
        # Carregando o áudio, forçando a taxa de amostragem desejada
        signal, sr = librosa.load(audio_path, sr=sampling_rate)

        # Normalizando o sinal para o intervalo [-1, 1]
        signal = signal / np.max(np.abs(signal))

        # Aplicando a Transformada Wavelet Contínua
        scales = np.arange(1, 128)  # Escalas de análise
        coefficients, _ = pywt.cwt(signal, scales, wavelet)

        # Gerando a figura do escalograma
        plt.figure(figsize=(5, 5), dpi=250)  # Tamanho e resolução
        plt.axis('on')                      # Remover eixos da imagem
        plt.imshow(np.abs(coefficients), aspect='auto', cmap='magma')
        plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
        plt.close()

        # Liberando memória após processamento
        gc.collect()

    except Exception as e:
        print(f"Erro ao processar '{audio_path}': {e}")


### Execução do Pipeline de Processamento

In [ ]:
# Percorrendo todas as categorias e subcategorias para processamento
for category in categories:
    for subcategory in subcategories:
        input_folder = os.path.join(base_path, category, subcategory)
        output_folder = os.path.join(output_base_path, category, subcategory)
        os.makedirs(output_folder, exist_ok=True)  # Garantir que o diretório de saída existe

        # Listando e processando todos os arquivos .wav
        for file in tqdm(os.listdir(input_folder), desc=f"Processando {category}/{subcategory}"):
            if file.endswith(".wav"):
                input_path = os.path.join(input_folder, file)
                output_path = os.path.join(output_folder, file.replace(".wav", ".png"))
                process_and_save_scalogram(input_path, output_path)

print("\nProcessamento concluído com sucesso!")


Processando validation/real: 100%|██████████| 1413/1413 [00:04<00:00, 290.86it/s]


Processamento concluído com sucesso!
